### import package

In [1]:
import os
from glob import glob
from easydict import EasyDict
from Analysis_template import AWS_s3
from Analysis_template import Model_template

import torch
from torch import nn, optim
import pytorch_lightning as pl
from torch.nn import functional as F
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader

In [2]:
from torchvision import datasets
import torchvision.transforms as transforms

### You need to revise only this part for your needs
### ↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓

### data preprocessing

In [3]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
train_data = datasets.CIFAR10('./data', train = True, download=True, transform = transform)
test_data = datasets.CIFAR10('./data', train = False, download=True, transform = transform)

Files already downloaded and verified
Files already downloaded and verified


### model define

In [5]:
class Model(Model_template) :
    def __init__(self, hyperperameters) :
        super().__init__(hyperparameters)
        
        self.loss = nn.CrossEntropyLoss()

        self.conv1 = nn.Conv2d(3, 32, 3)
        self.max_pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 16, 3)
        self.linear = nn.Linear(16 * 6 * 6, 10)
    
    def forward(self, x) :
        x = self.conv1(x)
        x = F.relu(self.max_pool(x))
        x = self.conv2(x)
        x = F.relu(self.max_pool(x))
        x = F.relu(self.linear(x.view(x.size(0), -1)))       
        
        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr)
        scheduler = StepLR(optimizer, step_size=self.hparams.step_size, 
                           gamma=self.hparams.gamma)
        
        return [optimizer], [scheduler]

### hyperparameters

In [6]:
 hyperparameters = EasyDict({'lr' : 0.007,
                            'max_epochs' :10,
                            'step_size' : 3, # scheduler
                            'gamma' : 0.9, # schduler
                            'batch_size' : 256, # train_batch_size
                            'test_batch_size' : -1, # test_batch_size
                            'gpus' : [0],
                            'num_workers' : 16,
                            'auto_lr_find' : False,
                            'save_top_k' : 3,
                            'folder' : 'best_model',
                             'early_stopping' : True,
                             'patience' : 5
                            })


if not os.path.isdir(hyperparameters['folder']) :
    os.mkdir(hyperparameters['folder'])

### model training

In [7]:
model = Model(hyperparameters)

In [8]:
model.fit(train_data, test_data)

EarlyStopping mode auto is unknown, fallback to auto mode.
EarlyStopping mode set to min for monitoring val_loss.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]



  | Name     | Type             | Params
----------------------------------------------
0 | loss     | CrossEntropyLoss | 0     
1 | conv1    | Conv2d           | 896   
2 | max_pool | MaxPool2d        | 0     
3 | conv2    | Conv2d           | 4 K   
4 | linear   | Linear           | 5 K   


epoch : 0, training loss : 2.1918, validation loss : 2.1133, ckpt_path = 200831_11:41:26/epoch=0_val_loss=2.1133


epoch : 1, training loss : 2.0972, validation loss : 2.0737, ckpt_path = 200831_11:41:26/epoch=1_val_loss=2.0737


epoch : 2, training loss : 2.0593, validation loss : 2.0578, ckpt_path = 200831_11:41:26/epoch=2_val_loss=2.0578


...

epoch : 6, training loss : 2.0021, validation loss : 2.0024, ckpt_path = 200831_11:41:26/epoch=6_val_loss=2.0024


epoch : 7, training loss : 1.9903, validation loss : 2.0083, ckpt_path = 200831_11:41:26/epoch=7_val_loss=2.0083


epoch : 8, training loss : 1.9873, validation loss : 2.0101, ckpt_path = 200831_11:41:26/epoch=8_val_loss=2.0101


Saving latest checkpoint..


epoch : 9, training loss : 1.9747, validation loss : 1.9899, ckpt_path = 200831_11:41:26/epoch=9_val_loss=1.9899



### model validation

In [9]:
def accuracy (y_hat, y) : # manual metric example
    return torch.sum(torch.max(y_hat, axis = 1)[1] == y).item()/len(y_hat)

In [10]:
model.test(test_data, 'accuracy', accuracy)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.2991}
--------------------------------------------------------------------------------



In [11]:
model.test(test_data, 'loss')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(2.0007, device='cuda:0')}
--------------------------------------------------------------------------------



### load model & validation

In [12]:
saved_folder_name = glob('./best_model/*')[0].split('/')[-1]

In [13]:
model = model.load_model(saved_folder_name)

succefully load : 200831_00:48:32/epoch=7_val_loss=1.9808


In [14]:
model.test(test_data, 'accuracy', accuracy)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.3081}
--------------------------------------------------------------------------------



In [15]:
model.test(test_data, 'loss')

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(1.9808, device='cuda:0')}
--------------------------------------------------------------------------------



### training Cross_validation

In [16]:
model.fit_cross_validation(train_data, 5, 0)

EarlyStopping mode auto is unknown, fallback to auto mode.
EarlyStopping mode set to min for monitoring val_loss.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


In [17]:
saved_folder_name = sorted(glob('./best_model/*'))[-1].split('/')[-1]

In [18]:
model.test_cross_validation(train_data, 'loss', saved_folder_name)

succefully load : 200831_00:49:55/1_fold/epoch=9_val_loss=1.7649


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(1.7649, device='cuda:0')}
--------------------------------------------------------------------------------

succefully load : 200831_00:49:55/2_fold/epoch=7_val_loss=1.8992


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(1.8992, device='cuda:0')}
--------------------------------------------------------------------------------

succefully load : 200831_00:49:55/3_fold/epoch=9_val_loss=1.8749


...

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(1.8749, device='cuda:0')}
--------------------------------------------------------------------------------

succefully load : 200831_00:49:55/4_fold/epoch=7_val_loss=1.2939


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(1.2939, device='cuda:0')}
--------------------------------------------------------------------------------

succefully load : 200831_00:49:55/5_fold/epoch=7_val_loss=2.0017


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'loss': tensor(2.0017, device='cuda:0')}
--------------------------------------------------------------------------------

